In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import os
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt

In [ ]:
model = tf.saved_model.load("ssd_mobilenet_v1_coco_2018_01_28")
model = model.signatures['serving_default']

In [ ]:
cwd = os.getcwd()
img_path = cwd+ "\\sample_input_image.jpg"
vid = cwd + "\\pedestrian_survaillance.mp4"

In [ ]:
def draw_bounding_boxes(model, image):
    imagenp = np.expand_dims(image, axis=0)
    imagenp = tf.convert_to_tensor(imagenp)
    p = model(imagenp)

    height = image.shape[0]
    width =  image.shape[1]
    font = cv2.FONT_HERSHEY_SIMPLEX 
    fontScale = 0.5
    for x in range(p['detection_boxes'][0].shape[0]):
        if int(sum(p['detection_boxes'][0][x].numpy())):
            ymin,xmin,ymax,xmax=p['detection_boxes'][0][x].numpy()

            (left, right, top, bottom) = (int(round((xmin * width))), int(round((xmax * width))), 
                                        int(round((ymin * height))), int(round((ymax * height))))

            res = cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 4) 
            res = cv2.putText(image, str(p['detection_scores'][0][x].numpy()), 
                       (left, top), font, fontScale, (255, 255, 255), 2, cv2.LINE_AA)
        
    return res

In [ ]:
# cv = cv2.VideoCapture(cwd+vid)
cv = cv2.VideoCapture(0)
while cv.isOpened():
    ret, frame = cv.read()
    if ret:
        f = draw_bounding_boxes(model, frame)
        cv2.imshow("Reading", f)
        if cv2.waitKey(1) == ord('q'):
            break
    else:
        break
cv.release()
cv2. destroyAllWindows() 

In [ ]:
imag = cv2.imread(img_path)
f = draw_bounding_boxes(model, imag)
plt.imshow(f)
plt.show()